# API Face Samples

## Objective
The script uses the Azure Face API to detect faces in an image, create and train a large face list and a large person group.

In [ ]:
import os
import uuid
from PIL import Image
from azure.core.credentials import AzureKeyCredential
from azure.ai.vision.face import FaceClient, FaceAdministrationClient
from azure.ai.vision.face.models import FaceDetectionModel, FaceRecognitionModel

def detect_faces(subscription_key, endpoint, image_path):
    with FaceClient(endpoint, AzureKeyCredential(subscription_key), headers = {"X-MS-AZSDK-Telemetry": "sample=create-face-collection"}) as face_client:
        with open(image_path, 'rb') as image_data:
            detected_faces = face_client.detect(
                    image_content=image_data.read(),
                    detection_model=FaceDetectionModel.DETECTION03,
                    recognition_model=FaceRecognitionModel.RECOGNITION04,
                    return_face_id=True,
                )
        return detected_faces

# Function to enlarge the bounding box
def enlarge_bounding_box(face_rectangle, image_width, image_height, enlargement_factor=1.2):
    left = max(0, face_rectangle.left - (face_rectangle.width * (enlargement_factor - 1) / 2))
    top = max(0, face_rectangle.top - (face_rectangle.height * (enlargement_factor - 1) / 2))
    width = min(image_width - left, face_rectangle.width * enlargement_factor)
    height = min(image_height - top, face_rectangle.height * enlargement_factor)
    return {'left': int(left), 'top': int(top), 'width': int(width), 'height': int(height)}

# Function to create a large face list, add faces to it, and train the list
def create_large_face_list(subscription_key, endpoint, large_face_list_id, faces, image_path, image_width, image_height):
    with FaceAdministrationClient(endpoint, AzureKeyCredential(subscription_key), headers = {"X-MS-AZSDK-Telemetry": "sample=create-face-collection"}) as face_admin_client:
        try:
            face_admin_client.large_face_list.get(large_face_list_id)
            print(f"Large face list ID {large_face_list_id} already exists")
        except Exception:
            face_admin_client.large_face_list.create(
                large_face_list_id,
                name="my_large_face_list",
                recognition_model=FaceRecognitionModel.RECOGNITION04
            )
            print(f"Large face list ID {large_face_list_id} created")
        
        # Add faces to the large face list
        face_count = 0
        for face in faces:
            face_rectangle = enlarge_bounding_box(face.face_rectangle, image_width, image_height)
            target_face = [face_rectangle['left'],face_rectangle['top'],face_rectangle['width'], face_rectangle['height']]
            with open(image_path, 'rb') as image:
                face_admin_client.large_face_list.add_face(
                    large_face_list_id,
                    image,
                    target_face=target_face,
                    detection_model=FaceDetectionModel.DETECTION03
                )
            face_count += 1
        print(f"Total faces added to large face list: {face_count}")

        # Train the large face list
        poller_face_list = face_admin_client.large_face_list.begin_train(
            large_face_list_id=large_face_list_id,
            polling_interval=5,
        )
        poller_face_list.wait()
        print(f"The large face list {large_face_list_id} is trained successfully.")

# Function to create a large person group, add persons to it, and train the group
def create_large_person_group(subscription_key, endpoint, large_person_group_id, faces, image_path, image_width, image_height):
    with FaceAdministrationClient(endpoint, AzureKeyCredential(subscription_key), headers = {"X-MS-AZSDK-Telemetry": "sample=create-face-collection"}) as face_admin_client:
        try:
            face_admin_client.large_person_group.get(large_person_group_id)
            print(f"Large person group ID {large_person_group_id} already exists")
        except Exception:
            face_admin_client.large_person_group.create(
                large_person_group_id,
                name="my_large_person_group",
                recognition_model=FaceRecognitionModel.RECOGNITION04
            )
            print(f"Large person group ID {large_person_group_id} created")
        
        # Add persons to the large person group
        person_count = 0
        for face in faces:
            person_id = face_admin_client.large_person_group.create_person(
                large_person_group_id,
                name="person_" + str(face.face_id),
            ).person_id
            face_rectangle = enlarge_bounding_box(face.face_rectangle, image_width, image_height)
            target_face = [face_rectangle['left'],face_rectangle['top'],face_rectangle['width'], face_rectangle['height']]
            with open(image_path, 'rb') as image:
                face_admin_client.large_person_group.add_face(
                    large_person_group_id,
                    person_id,
                    image,
                    target_face=target_face,
                    detection_model=FaceDetectionModel.DETECTION03
                )
            person_count += 1
        print(f"Total persons added to large person group: {person_count}")

        # Train the large person group
        poller_person_group = face_admin_client.large_person_group.begin_train(
            large_person_group_id=large_person_group_id,
            polling_interval=5,
        )
        poller_person_group.wait()
        print(f"The large person group {large_person_group_id} is trained successfully.")

# Function to get image dimensions
def get_image_dimensions(image_path):
    with Image.open(image_path) as img:
        return img.width, img.height

# Replace with your Azure Face API subscription key and endpoint
FACE_KEY = os.environ["FACE_API_KEY"]
FACE_ENDPOINT = os.environ["FACE_ENDPOINT_URL"]
image_path = 'path_to_image.jpg'

# Detect faces in the image
faces = detect_faces(FACE_KEY, FACE_ENDPOINT, image_path)

# Create a unique large face list ID and large person group ID
large_face_list_id = str(uuid.uuid4())
large_person_group_id = str(uuid.uuid4())

image_width, image_height = get_image_dimensions(image_path)

# Create a large face list and add faces to it
create_large_face_list(FACE_KEY, FACE_ENDPOINT, large_face_list_id, faces, image_path, image_width, image_height)

# Create a large person group, add persons to it, and train the group
create_large_person_group(FACE_KEY, FACE_ENDPOINT, large_person_group_id, faces, image_path, image_width, image_height)

print(f"Large face list ID: {large_face_list_id}")
print(f"Large person group ID: {large_person_group_id}")